# 03 - Federated Learning

## Defines

Define the available types of federated learning.

 - 'STRATIFIED': Stratified sampling of the data. The data is split into a number of shards, and each shard is assigned to a client. The data is split in a stratified manner, meaning that the distribution of the labels is approximately the same in each shard.
 - 'MISSING_1_ATTACK' - Each client is assigned a shard of data, each shard is missing one of the attack labels. Other clients in the network are exposed to the attack label, but the specific client is not. This demonstrates the ability of federated learning to protect against unknown attacks.
 - '1_ATTACK_ONLY' - Each client is assigned a shard of data, each shard contains only one of the attack labels.
 - 'HALF_BENIGN_ONLY' - Half of the clients are exposed to Benign data only, the other half are exposed to all data.


In [49]:
### THIS SECTION NEEDS TO BE SET TO DETERMINE WHICH CONFIGURATION METHOD TO UTILISE

SPLIT_AVAILABLE_METHODS = ['STRATIFIED','MISSING_1_ATTACK', '1_ATTACK_ONLY', 'HALF_BENIGN_ONLY' ]
METHOD = 'HALF_BENIGN_ONLY'
NUM_OF_STRATIFIED_CLIENTS = 10  # only applies to stratified method
NUM_OF_ROUNDS = 10              # Number of FL rounds


The above test method in conjunction with the below classification selection will determine the number of clients.

EG: 
`STRATIFIED` with:
 - `ALL TYPES` - Results in `NUM_OF_STRATIFIED_CLIENTS` clients. Each client will have a stratified sample of the data.

`MISSING_1_ATTACK` with:
 - `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 32 attack labels.
 - `group_classifier` - Results in 7 clients. Each client will have benign traffic and 6 attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels.

`1_ATTACK_ONLY` with:
 - `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 1 attack label.
 - `group_classifier` - Results in 7 clients. Each client will have benign traffic and 1 attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS MISSING_1_ATTACK for binary classifier

`HALF_BENIGN_ONLY` with:
 - `individual_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 33 malicious attack labels.
 - `group_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 7 malicious attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS MISSING_1_ATTACK for binary classifier


In [50]:
individual_classifier = True
group_classifier = False
binary_classifier = False


Include the defines for the dataframe columns and the attack labels and their mappings

In [51]:
from enum import Enum
from includes import *

##  Imports

In [52]:
%%capture
%pip install flwr[simulation] torch torchvision matplotlib sklearn openml

In [53]:
import os
import pandas as pd
import numpy as np
import flwr as fl
from tqdm import tqdm
import warnings
#warnings.filterwarnings('ignore')

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split


In [54]:
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

flwr 1.4.0
numpy 1.24.2
torch 1.13.1
Training on cuda:0


## Load the Dataset

In [55]:
DATASET_DIRECTORY = '../datasets/CICIoT2023/'

## Training data

Either read the training pickle file if it exists, or process the dataset from scratch.

In [56]:
# Check to see if the file 'training_data.pkl' exists in the directory. If it does, load it. If not, print an error.
if os.path.isfile('training_data.pkl'):
    print("File exists, loading data...")
    train_df = pd.read_pickle('training_data.pkl')
    print("Training data loaded from pickle file.")

else:
    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    # Print the number of files in each set
    print('Training sets: {}'.format(len(training_sets)))
    print('Test sets: {}'.format(len(test_sets)))

    # ######################
    # # HACK TEMP CODE
    # ######################
    # # Set training_sets to the last entry of training_sets
    # training_sets = training_sets[-33:]
    # print(f"HACK TO REPLICATE ORIGINAL AUTHORS CODE WITH ONE FILE TRAIN - {training_sets}")
    # #####################
    # # HACK END TEMP CODE
    # ######################

    # Concatenate all training sets into one dataframe
    dfs = []
    print("Reading training data...")
    for train_set in tqdm(training_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + train_set)
        dfs.append(df_new)
    train_df = pd.concat(dfs, ignore_index=True)

    # Complete training data set size
    print("Complete training data size: {}".format(train_df.shape))

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    train_df['label'] = train_df['label'].map(dict_34_classes)

    # The training data is the 80% of the CSV files in the dataset. The test data is the remaining 20%.
    # The Ray Federated learning mechanism cannot cope with all of the 80% training data, so we will split
    # the training data using test_train_split. The test data will be ignored as we will use all the data 
    # from the train_sets files as our training data to keep parity with the original authors code.
    # 
    # By using a subset of the training data split this way, we can have a randomised selection of data
    # from all the training CSV files, stratified by the attack types.
    
    # Percentage of original training data to use.
    TRAIN_SIZE = 0.025
    
    print(f"Splitting the data into {TRAIN_SIZE*100}%")
    
    X_train, X_test, y_train, y_test = train_test_split(train_df[X_columns], train_df[y_column], test_size= (1 - TRAIN_SIZE), random_state=42, stratify=train_df[y_column])

    # Recombine X_train, and y_train into a dataframe
    train_df = pd.concat([X_train, y_train], axis=1)

    # Clean up unused variables

    del X_train, y_train, X_test, y_test
    
    # Save the output to a pickle file
    print("Writing training data to pickle file...")
    train_df.to_pickle('training_data.pkl')

print("Training data size: {}".format(train_df.shape))


File exists, loading data...
Training data loaded from pickle file.
Training data size: (908660, 47)


In [57]:
# show the unique values counts in the label column for train_df
print("Counts of attacks in train_df:")
print(train_df['label'].value_counts())

Counts of attacks in train_df:
6     140143
4     105397
5      87536
2      79671
3      78959
1      78729
7      70063
13     64601
15     51987
14     39474
0      21372
17     19294
19     17330
18     14637
10      8794
26      5990
9       5595
8       5560
25      3478
24      2615
21      1914
22      1596
16      1403
23       729
12       559
11       454
33       252
27       115
32       106
31       102
29        74
28        62
20        44
30        25
Name: label, dtype: int64


In [58]:
train_df

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
13070266,0.000000,0.00,1.00,64.00,60.072672,60.072672,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.312483e+07,9.5,9.165151,0.000000,0.000000,0.00,141.55,6
11111392,0.032670,162.67,16.89,65.91,0.685361,0.685361,0.0,0.0,0.0,0.0,...,6.433546,160.92,8.301119e+07,9.5,17.761121,9.106583,233.739318,0.19,141.55,13
14955759,0.000000,0.00,1.00,64.00,61.182930,61.182930,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.315025e+07,9.5,9.165151,0.000000,0.000000,0.00,141.55,6
3245562,0.092809,63.12,5.94,62.90,1.437224,1.437224,0.0,0.0,0.0,0.0,...,0.403326,54.10,8.331381e+07,9.5,10.408309,0.571145,0.764002,0.24,141.55,2
20485085,2.581335,228.96,6.00,64.00,1.754608,1.754608,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.336144e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28314625,0.000624,54.86,6.00,64.00,144.071746,144.071746,0.0,0.0,1.0,0.0,...,0.997117,54.20,8.309440e+07,9.5,10.425904,1.411650,6.126863,0.17,141.55,3
25072079,0.000000,162.00,17.00,64.00,45.678888,45.678888,0.0,0.0,0.0,0.0,...,0.000000,162.00,8.300816e+07,9.5,18.000000,0.000000,0.000000,0.00,141.55,13
19742459,0.057148,17765.00,17.00,64.00,12286.507156,12286.507156,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.310670e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,4
27891842,0.000000,56.23,6.00,64.00,2.046859,2.046859,0.0,0.0,0.0,0.0,...,1.340412,55.57,8.294319e+07,9.5,10.424976,1.906483,43.601168,0.05,141.55,15


---
## Test Data
Concat the test data into a single dataframe

In [59]:
# Check to see if the file 'test_data.pkl' exists in the directory. If it does, load it. If not, print an error.
testing_data_pickle_file = 'testing_data.pkl'

if os.path.isfile(testing_data_pickle_file):
    print(f"File {testing_data_pickle_file} exists, loading data...")
    test_df = pd.read_pickle(testing_data_pickle_file)
    print("Test data loaded from pickle file.")

else:
    print(f"File {testing_data_pickle_file} does not exist, constructing data...")

    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    ############################################
    ############################################
    # HACK - Make things quicker for now
    ############################################
    ############################################

    # test_sets = df_sets[int(len(df_sets)*.95):]
    
    # # Set training_sets to the last entry of training_sets
    # test_sets = test_sets[-2:]
    
    ############################################
    ############################################
    # END HACK 
    ############################################
    ############################################

    # Print the number of files in each set
    print('Test sets: {}'.format(len(test_sets)))
    
    # Concatenate all testing sets into one dataframe
    dfs = []
    print("Reading test data...")
    for test_set in tqdm(test_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + test_set)
        dfs.append(df_new)
    test_df = pd.concat(dfs, ignore_index=True)

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    test_df['label'] = test_df['label'].map(dict_34_classes)

    # Save the output to a pickle file
    print(f"Writing test data to pickle file {testing_data_pickle_file}...")
    test_df.to_pickle(testing_data_pickle_file)

print("Testing data size: {}".format(test_df.shape))

File testing_data.pkl exists, loading data...
Test data loaded from pickle file.
Testing data size: (10340161, 47)


In [60]:
print("Number of rows in train_df: {}".format(len(train_df)))
print("Number of rows in test_df: {}".format(len(test_df)))

train_size = len(train_df)
test_size = len(test_df)

Number of rows in train_df: 908660
Number of rows in test_df: 10340161


---
# Scale the test and train data

### Scale the training data input features

In [61]:
scaler = StandardScaler()
train_df[X_columns] = scaler.fit_transform(train_df[X_columns])

### Scale the testing data input features

In [62]:
test_df[X_columns] = scaler.fit_transform(test_df[X_columns])

---
# Define the classification problem - (2 classes, 8 classes or 34 classes)
Change the following cell to select the classification type

If the METHOD == STRATIFIED, then we can use any classifier
If the METHOD == ATTACK_GROUP then we must use Group Classifier.

In [63]:

class_size_map = {2: "Binary", 8: "Group", 34: "Individual"}

if group_classifier:
    print("Group 8 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_7_classes values
    test_df['label'] = test_df['label'].map(dict_8_classes)
    train_df['label'] = train_df['label'].map(dict_8_classes)
    class_size = "8"      
    
elif binary_classifier:
    print("Binary 2 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_2_classes values
    test_df['label'] = test_df['label'].map(dict_2_classes)
    train_df['label'] = train_df['label'].map(dict_2_classes)
    class_size = "2"

else:
    print ("Individual 34 Class classifier... - No adjustments to labels in test and train dataframes")
    class_size = "34"


Individual 34 Class classifier... - No adjustments to labels in test and train dataframes


---
# Split the Training Data into partitions for the Federated Learning clients depending on the test required
As a reminder:

`STRATIFIED` with:
 - `ALL TYPES` - Results in `NUM_OF_STRATIFIED_CLIENTS` clients. Each client will have a stratified sample of the data.

`MISSING_1_ATTACK` with:
 - `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 32 attack labels.
 - `group_classifier` - Results in 7 clients. Each client will have benign traffic and 6 attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels.

`1_ATTACK_ONLY` with:
 - `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 1 attack label.
 - `group_classifier` - Results in 7 clients. Each client will have benign traffic and 1 attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS MISSING_1_ATTACK for binary classifier

`HALF_BENIGN_ONLY` with:
 - `individual_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 33 malicious attack labels.
 - `group_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 7 malicious attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels. - SAME AS MISSING_1_ATTACK for binary classifier


In [64]:
from sklearn.model_selection import StratifiedKFold

# Define fl_X_train and fl_y_train
fl_X_train = []
fl_y_train = []

client_df = pd.DataFrame()

if METHOD == 'STRATIFIED':
    print(f"{Colours.YELLOW.value}STRATIFIED METHOD{Colours.NORMAL.value} with {class_size} class classifier")
    # We are going to split the training data into 'NUM_OF_STRATIFIED_CLIENTS' smaller groups using StratifiedKFold
    skf = StratifiedKFold(n_splits=NUM_OF_STRATIFIED_CLIENTS, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(train_df[X_columns], train_df[y_column]):
        fl_X_train.append(train_df[X_columns].iloc[test_index])
        fl_y_train.append(train_df[y_column].iloc[test_index])

elif METHOD == 'MISSING_1_ATTACK':
    print(f"{Colours.YELLOW.value}MISSING_1_ATTACK METHOD{Colours.NORMAL.value} with {class_size} class classifier")

    if individual_classifier or group_classifier:
        # Set the number of splits required to the number of classes - 1
        num_splits = int(class_size) - 1
    else:
        # For binary classifier, set the number of splits to 10
        num_splits = 10

    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

    # When creating the clients, we will remove one attack class from the training data
    # For the binary classifier, evey other client will have the benign class removed
    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        if binary_classifier:
            print(f"i: {i} = i % 2 = {i % 2}")
            if i % 2 == 0:
                print("Benign only")
                # Create a new dataframe for the client data with only benign traffic
                client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != 1]], ignore_index=True)
                fl_X_train.append(client_df[X_columns])
                fl_y_train.append(client_df[y_column])
            else:
                print("Both")
                # Create a new dataframe for the client data
                fl_X_train.append(train_df[X_columns].iloc[test_index])
                fl_y_train.append(train_df[y_column].iloc[test_index])
        else:
            # Create a new dataframe for the client data
            client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i+1]], ignore_index=True)
            fl_X_train.append(client_df[X_columns])
            fl_y_train.append(client_df[y_column])

elif METHOD == '1_ATTACK_ONLY':
    print(f"{Colours.YELLOW.value}1_ATTACK_ONLY METHOD{Colours.NORMAL.value} with {class_size} class classifier")
    # Each client only has one attack class in their training data along with the Benign data
    
    if individual_classifier or group_classifier:
        # Set the number of splits required to the number of classes - 1
        num_splits = int(class_size) - 1
    else:
        # For binary classifier, set the number of splits to 10
        num_splits = 10

    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

    # When creating the clients, we will only add the benign data and the attack class for that client
    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        if binary_classifier:
            print(f"i: {i} = i % 2 = {i % 2}")
            if i % 2 == 0:
                print("Benign only")
                # Create a new dataframe for the client data with only benign traffic
                client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != 1]], ignore_index=True)
                fl_X_train.append(client_df[X_columns])
                fl_y_train.append(client_df[y_column])
            else:
                print("Both")
                # Create a new dataframe for the client data
                fl_X_train.append(train_df[X_columns].iloc[test_index])
                fl_y_train.append(train_df[y_column].iloc[test_index])
        else:
            # Create a new dataframe for the client data
            client_df = pd.concat([train_df.iloc[test_index][(train_df[y_column] == 0) | (train_df[y_column] == i+1)]], ignore_index=True)
            fl_X_train.append(client_df[X_columns])
            fl_y_train.append(client_df[y_column])

elif METHOD == 'HALF_BENIGN_ONLY':
    print(f"{Colours.YELLOW.value}HALF_BENIGN_ONLY METHOD{Colours.NORMAL.value} with {class_size} class classifier")

    num_splits = 10

    # Split into 10 client data
    skf = StratifiedKFold(n_splits=NUM_OF_STRATIFIED_CLIENTS, shuffle=True, random_state=42)

    # For i % 2 == 0, add only benign data
    # For i % 2 == 1, add all data
    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        if i % 2 == 0:
            print("Benign only")
            # Create a new dataframe for the client data with only benign traffic
            client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] == 0]], ignore_index=True)
            fl_X_train.append(client_df[X_columns])
            fl_y_train.append(client_df[y_column])
        else:
            print("All Classes")
            fl_X_train.append(train_df[X_columns].iloc[test_index])
            fl_y_train.append(train_df[y_column].iloc[test_index])
else:
    print(f"{Colours.RED.value}ERROR: Method {METHOD} not recognised{Colours.NORMAL.value}")
    exit()



HALF_BENIGN_ONLY METHOD with 34 class classifier
Benign only
All Classes


C:\Users\Jon\AppData\Local\Temp\ipykernel_29564\2411292254.py:99: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] == 0]], ignore_index=True)


Benign only
All Classes


C:\Users\Jon\AppData\Local\Temp\ipykernel_29564\2411292254.py:99: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] == 0]], ignore_index=True)


Benign only
All Classes


C:\Users\Jon\AppData\Local\Temp\ipykernel_29564\2411292254.py:99: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] == 0]], ignore_index=True)


Benign only
All Classes


C:\Users\Jon\AppData\Local\Temp\ipykernel_29564\2411292254.py:99: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] == 0]], ignore_index=True)


Benign only
All Classes


C:\Users\Jon\AppData\Local\Temp\ipykernel_29564\2411292254.py:99: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] == 0]], ignore_index=True)


In [65]:
NUM_OF_CLIENTS = len(fl_X_train)

for i in range(len(fl_X_train)):
    # Show the unique values in the y column
    (f"Client ID: {i}")
    print(f"fl_X_train[{i}].shape: {fl_X_train[i].shape}")  
    print(f"fl_y_train[{i}].value_counts():\n{fl_y_train[i].value_counts()}")
    print(f"fl_y_train[{i}].unique(): {fl_y_train[i].unique()}\n")

# Check that fl_X_train[0] and fl_X_train[1] contain different data
print(f"fl_X_train[0].equals(fl_X_train[1]): {fl_X_train[0].equals(fl_X_train[1])}")

fl_X_train[0].shape: (2138, 46)
fl_y_train[0].value_counts():
0    2138
Name: label, dtype: int64
fl_y_train[0].unique(): [0]

fl_X_train[1].shape: (90866, 46)
fl_y_train[1].value_counts():
6     14015
4     10540
5      8753
2      7967
3      7896
1      7872
7      7006
13     6460
15     5199
14     3947
0      2137
17     1930
19     1733
18     1464
10      880
26      599
9       560
8       556
25      348
24      261
21      191
22      160
16      140
23       73
12       56
11       45
33       26
27       12
31       10
32       10
29        7
28        7
20        4
30        2
Name: label, dtype: int64
fl_y_train[1].unique(): [ 7  4  0  3  6  5 14  1 13 15  2 10 16 19  9 18 11 17 25 24 26  8 22 21
 12 23 29 33 27 31 28 32 20 30]

fl_X_train[2].shape: (2137, 46)
fl_y_train[2].value_counts():
0    2137
Name: label, dtype: int64
fl_y_train[2].unique(): [0]

fl_X_train[3].shape: (90866, 46)
fl_y_train[3].value_counts():
6     14014
4     10540
5      8753
2      7967
3      7

Prepare an output directory where we can store the results of the federated learning

In [66]:
# Create an "Output" directory if it doesnt exist already
if not os.path.exists("Output"):
    os.makedirs("Output")

sub_dir_name = f"train_size-{train_size}_test_size-{test_size}"

# if sub_dir_name does not exist, create it
if not os.path.exists(f"Output/{sub_dir_name}"):
    os.makedirs(f"Output/{sub_dir_name}")

test_directory_name = f"{METHOD}_Classifier-{class_size}_Clients-{NUM_OF_CLIENTS}"

# Create an "Output/{METHOD}-{NUM_OF_CLIENTS}-{NUM_OF_ROUNDS}" directory if it doesnt exist already
if not os.path.exists(f"Output/{sub_dir_name}/{test_directory_name}"):
    os.makedirs(f"Output/{sub_dir_name}/{test_directory_name}")

# Ensure the directory is empty
for file in os.listdir(f"Output/{sub_dir_name}/{test_directory_name}"):
    file_path = os.path.join(f"Output/{sub_dir_name}/{test_directory_name}", file)
    if os.path.isfile(file_path):
        os.unlink(file_path)

# Original training size is the sum of all the fl_X_train sizes
original_training_size = 0
for i in range(len(fl_X_train)):
    original_training_size += fl_X_train[i].shape[0]

# Write this same info to the output directory/Class Split Info.txt
with open(f"Output/{sub_dir_name}/{test_directory_name}/Class Split Info.txt", "w") as f:
    for i in range(len(fl_X_train)):
        f.write(f"Client ID: {i}\n")
        f.write(f"fl_X_train.shape: {fl_X_train[i].shape}\n")
        f.write(f"Training data used {original_training_size}")
        f.write(f"fl_y_train.value_counts():\n{fl_y_train[i].value_counts()}\n")
        f.write(f"fl_y_train.unique(): {fl_y_train[i].unique()}\n\n")

### Convert the training dataset

In [67]:
# Convert the testing daya to X_test and y_test ndarrays
X_test = test_df[X_columns].to_numpy()
y_test = test_df[y_column].to_numpy()

In [68]:
num_unique_classes = len(train_df[y_column].unique())

train_df_shape = train_df.shape
test_df_shape = test_df.shape

# We are now done with the train_df and test_df dataframes, so we can delete them to free up memory
del train_df
del test_df
del client_df

---
### Data check

In [69]:
print("NUM_CLIENTS:", NUM_OF_CLIENTS)

print("NUM_ROUNDS:", NUM_OF_ROUNDS)
print()


print("Original training size: {}".format(original_training_size))


print("Checking training data split groups")
for i in range(len(fl_X_train)):
    print(i, ":", "X Shape", fl_X_train[i].shape, "Y Shape", fl_y_train[i].shape)


# Print the sizes of X_test and y_test
print("\nChecking testing data")
print("X_test size: {}".format(X_test.shape))
print("y_test size: {}".format(y_test.shape))

print("\nDeploy Simulation")

NUM_CLIENTS: 10
NUM_ROUNDS: 10

Original training size: 465016
Checking training data split groups
0 : X Shape (2138, 46) Y Shape (2138,)
1 : X Shape (90866, 46) Y Shape (90866,)
2 : X Shape (2137, 46) Y Shape (2137,)
3 : X Shape (90866, 46) Y Shape (90866,)
4 : X Shape (2137, 46) Y Shape (2137,)
5 : X Shape (90866, 46) Y Shape (90866,)
6 : X Shape (2137, 46) Y Shape (2137,)
7 : X Shape (90866, 46) Y Shape (90866,)
8 : X Shape (2137, 46) Y Shape (2137,)
9 : X Shape (90866, 46) Y Shape (90866,)

Checking testing data
X_test size: (10340161, 46)
y_test size: (10340161,)

Deploy Simulation


----
# Federated Learning
## Import the libraries and print the versions

In [70]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout



Define the Client and Server code

In [71]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

print('scikit-learn {}.'.format(sklearn.__version__))
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("tf", tf.__version__)
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

import datetime

client_evaluations = []

class NumpyFlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, train_data, train_labels):
        self.model = model
        self.cid = cid
        self.train_data = train_data
        self.train_labels = train_labels

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Training...")
        self.model.fit(self.train_data, self.train_labels, epochs=5, batch_size=32)
        print ("Client ", self.cid, "Training complete...")
        return self.model.get_weights(), len(self.train_data), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Evaluating...")
        loss, accuracy = self.model.evaluate(self.train_data, self.train_labels, batch_size=32)
        print(f"{Colours.YELLOW.value}Client {self.cid} evaluation complete - Accuracy: {accuracy:.6f}, Loss: {loss:.6f}{Colours.NORMAL.value}")

        # Write the same message to the "Output/{cid}_Evaluation.txt" file
        with open(f"Output/{sub_dir_name}/{test_directory_name}/{self.cid}_Evaluation.txt", "a") as f:
            f.write(f"{datetime.datetime.now()} - Client {self.cid} evaluation complete - Accuracy: {accuracy:.6f}, Loss: {loss:.6f}\n")

            # Close the file
            f.close()

        return loss, len(self.train_data), {"accuracy": accuracy}
    
    def predict(self, incoming):
        prediction = np.argmax( self.model.predict(incoming) ,axis=1)
        return prediction

def client_fn(cid: str) -> NumpyFlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    #model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    print ("Client ID:", cid)

    model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(50, activation='relu'),  
      Dense(25, activation='relu'),  
      Dense(num_unique_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

   
    partition_id = int(cid)
    X_train_c = fl_X_train[partition_id]
    y_train_c = fl_y_train[partition_id]

    # Create a  single Flower client representing a single organization
    return NumpyFlowerClient(cid, model, X_train_c, y_train_c)


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
eval_count = 0

def get_evaluate_fn(server_model):
    global eval_count
    """Return an evaluation function for server-side evaluation."""
    # The `evaluate` function will be called after every round
    
    
    def evaluate(server_round, parameters, config):
        global eval_count
        
        # Update model with the latest parameters
        server_model.set_weights(parameters)
        print (f"Server Evaluating... Evaluation Count:{eval_count}")
        loss, accuracy = server_model.evaluate(X_test, y_test)
        
        y_pred = server_model.predict(X_test)
        print ("Prediction: ", y_pred, y_pred.shape)
        #cmatrix = confusion_matrix(y_test, np.rint(y_pred))
        #print ("confusion_matrix:", cmatrix, cmatrix.shape)
                        
        print(f"{Colours.YELLOW.value}Server evaluation complete - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}{Colours.NORMAL.value}")

        # Write the same message to the "Output/Server_Evaluation.txt" file
        with open(f"Output/{sub_dir_name}/{test_directory_name}/Server_Evaluation.txt", "a") as f:
            f.write(f"{datetime.datetime.now()} - {server_round} : Server evaluation complete - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}\n")

            # Close the file
            f.close()
        
        np.save("y_pred-" + str(eval_count) + ".npy", y_pred)
        #np.save("cmatrix-" + str(eval_count) + ".npy", cmatrix)
        eval_count = eval_count + 1
        
        return loss, {"accuracy": accuracy}
    return evaluate



server_model = Sequential([
    #Flatten(input_shape=(79,1)),
    Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
    Dense(50, activation='relu'),  
    Dense(25, activation='relu'),  
    Dense(num_unique_classes, activation='softmax')
])


server_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2, #10,
        min_evaluate_clients=2, #5,
        min_available_clients=2, #10,
        evaluate_fn=get_evaluate_fn(server_model),
        #evaluate_metrics_aggregation_fn=weighted_average,
)

scikit-learn 1.2.0.
flwr 1.4.0
numpy 1.24.2
tf 2.11.0


In [72]:
%%time
print (f"{Colours.YELLOW.value}\nDeploy simulation... Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier")
print (f"Number of Clients = {NUM_OF_CLIENTS}\n")
print (f"Writing output to: {sub_dir_name}/{test_directory_name}\n{Colours.NORMAL.value}")

# Output the same information to the Output/Run_details.txt file
with open(f"Output/{sub_dir_name}/{test_directory_name}/Run_details.txt", "a") as f:
    f.write(f"{datetime.datetime.now()} - Deploy simulation... Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier\n")
    f.write(f"{datetime.datetime.now()} - Number of Clients = {NUM_OF_CLIENTS}\n")

    # Write Original train_df size
    f.write(f"{datetime.datetime.now()} - Original train_df size: {train_df_shape}\n")

    # Write the training data split groups
    for i in range(len(fl_X_train)):
        f.write(f"{datetime.datetime.now()} - {i}: X Shape {fl_X_train[i].shape}, Y Shape {fl_y_train[i].shape}\n")

    # Write the testing data
    f.write(f"{datetime.datetime.now()} - X_test size: {X_test.shape}\n")
    f.write(f"{datetime.datetime.now()} - y_test size: {y_test.shape}\n")
    
# close the file
f.close()

start_time = datetime.datetime.now()

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_OF_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_OF_ROUNDS),
    strategy=strategy,
)

end_time = datetime.datetime.now()
print("Total time taken: ", end_time - start_time)

print (f"{Colours.YELLOW.value} SIMULATION COMPLETE. Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier")
print (f"Number of Clients = {NUM_OF_CLIENTS}{Colours.NORMAL.value}\n")

# Output the same information to the Output/Run_details.txt file
with open(f"Output/{sub_dir_name}/{test_directory_name}/Run_details.txt", "a") as f:
    f.write(f"{datetime.datetime.now()} - SIMULATION COMPLETE. Method = {METHOD} - {class_size_map[num_unique_classes]} ({class_size}) Classifier\n")
    f.write(f"{datetime.datetime.now()} - Total time taken: {end_time - start_time}\n")

INFO flwr 2023-07-13 07:34:48,359 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)



Deploy simulation... Method = HALF_BENIGN_ONLY - Individual (34) Classifier
Number of Clients = 10

Writing output to: train_size-908660_test_size-10340161/HALF_BENIGN_ONLY_Classifier-34_Clients-10

(launch_and_evaluate pid=38236) Client  11 Evaluating... [repeated 16x across cluster]
(launch_and_evaluate pid=38236) Client ID: 11 [repeated 15x across cluster]
(launch_and_evaluate pid=7376)  [repeated 35x across cluster]
(launch_and_evaluate pid=7376)   1/153 [..............................] - ETA: 31s - loss: 0.4616 - accuracy: 1.0000 [repeated 5x across cluster]
(launch_and_evaluate pid=7376) 153/153 [==============================] - 0s 915us/step - loss: 0.5501 - accuracy: 0.9953 [repeated 16x across cluster]
(launch_and_evaluate pid=42948)  1/21 [>.............................] - ETA: 4s - loss: 1.4156e-07 - accuracy: 1.0000 [repeated 11x across cluster]
(launch_and_evaluate pid=22988)  95/104 [==========================>...] - ETA: 0s - loss: 1.5675 - accuracy: 0.1980
(launch_and

2023-07-13 07:34:57,385	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-07-13 07:35:10,789 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 24.0, 'object_store_memory': 13598828544.0, 'node:127.0.0.1': 1.0, 'memory': 27197657088.0, 'GPU': 1.0}
INFO flwr 2023-07-13 07:35:10,794 | server.py:86 | Initializing global parameters
INFO flwr 2023-07-13 07:35:10,799 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2023-07-13 07:35:16,568 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-07-13 07:35:16,575 | server.py:88 | Evaluating initial parameters


(launch_and_get_parameters pid=33776) Client ID: 8
Server Evaluating... Evaluation Count:0
323131/323131 [==============================] - 385s 1ms/step
Prediction:  [[0.03054304 0.02356081 0.00976919 ... 0.02706297 0.05161174 0.01412552]
 [0.0257693  0.03123254 0.02256867 ... 0.02923803 0.03084666 0.02933784]
 [0.02476864 0.05183615 0.00559102 ... 0.04210653 0.02604311 0.01043699]
 ...
 [0.03449262 0.03220236 0.02421469 ... 0.03079553 0.02625427 0.02509729]
 [0.02848698 0.03128928 0.02637969 ... 0.03135136 0.02756293 0.02632673]
 [0.033593   0.03255452 0.01533277 ... 0.03441405 0.03266612 0.02414278]] (10340161, 34)
Server evaluation complete - Accuracy: 0.0938, Loss: 3.7124


INFO flwr 2023-07-13 07:50:19,173 | server.py:91 | initial parameters (loss, other metrics): 3.7123847007751465, {'accuracy': 0.09377126395702362}
INFO flwr 2023-07-13 07:50:19,181 | server.py:101 | FL starting
DEBUG flwr 2023-07-13 07:50:19,188 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)


...

Server Evaluating... Evaluation Count:7
167495/323131 [==============>...............] - ETA: 5:01 - loss: 0.0817 - accuracy: 0.9770

2023-07-13 09:45:56,143	ERROR import_thread.py:80 -- ImportThread: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Stream removed"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:54868 {grpc_message:"Stream removed", grpc_status:2, created_time:"2023-07-13T08:45:56.144172568+00:00"}"
>
Exception in thread ray_listen_error_messages:
Traceback (most recent call last):
  File "c:\Users\Jon\anaconda3\envs\py310copy\lib\threading.py", line 1016, in _bootstrap_inner
2023-07-13 09:45:56,151	ERROR worker.py:914 -- print_logs: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Stream removed"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:54868 {created_time:"2023-07-13T08:45:56.144285019+00:00", grpc_status:2, grpc_message:"Stream removed"}"
>
    self.run()
  File "c:\Users\Jon\anaconda3\envs\py310copy\lib\threading.py", line 953, in run


167497/323131 [==============>...............] - ETA: 5:01 - loss: 0.0817 - accuracy: 0.9770

    self._target(*self._args, **self._kwargs)
  File "c:\Users\Jon\anaconda3\envs\py310copy\lib\site-packages\ray\_private\worker.py", line 2004, in listen_error_messages
    _, error_data = worker.gcs_error_subscriber.poll()
  File "c:\Users\Jon\anaconda3\envs\py310copy\lib\site-packages\ray\_private\gcs_pubsub.py", line 300, in poll
    self._poll_locked(timeout=timeout)
  File "c:\Users\Jon\anaconda3\envs\py310copy\lib\site-packages\ray\_private\gcs_pubsub.py", line 217, in _poll_locked
    fut.result(timeout=1)
  File "c:\Users\Jon\anaconda3\envs\py310copy\lib\site-packages\grpc\_channel.py", line 744, in result
    raise self
grpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Stream removed"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:54868 {grpc_message:"Stream removed", grpc_status:2, created_time:"2023-07-13T08:45:56.14427499+00:00"}"
>


323131/323131 [==============================] - 682s 2ms/step - loss: 0.0823 - accuracy: 0.9771
